# The TinyALU TLM Testbench

In [ ]:
from pyuvm import *

class Producer(uvm_component):
    def build_phase(self):
        self.pp = uvm_put_port("pp", self)
    
    def run_phase(self):
        self.raise_objection()
        for nn in range(5):
            self.pp.put(nn)
            print(f"Put {nn}", end=" ")
        self.drop_objection()
        
class Consumer(uvm_component):
    def build_phase(self):
        self.gp = uvm_get_port("gp", self)
    
    def run_phase(self):
        while True:
            nn = self.gp.get()
            print(f"Got {nn}", end=" ")

class FIFOTest(uvm_test):
    def build_phase(self):
        self.prod = Producer("prod", self)
        self.cons = Consumer("cons", self)
        self.fifo = uvm_tlm_fifo("fifo", self)
        
    def end_of_elaboration_phase(self):
        file_handler = logging.FileHandler("put.txt")
        put_logger = logging.getLogger("uvm_test_top.fifo.put_export")
        put_logger.addHandler(file_handler)
        self.set_logging_level_hier(FIFO_DEBUG)
    
    def connect_phase(self):
        self.prod.pp.connect(self.fifo.put_export)
        self.cons.gp.connect(self.fifo.get_export)

uvm_root().run_test("FIFOTest")